In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 55 kB 1.7 MB/s 
     |████████████████████████████████| 8.2 MB 6.1 MB/s 
     |████████████████████████████████| 371 kB 28.2 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=847368 sha256=0513062ad7f92befc42021d1786dcda654f38f9286fc21279d19f33bd113cbe9
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [3]:
import random
import re
import string
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor
import warnings 
from sklearn.linear_model import LogisticRegression
from PIL import Image
from IPython.display import clear_output
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from typing import List
from sklearn.metrics import classification_report
import pickle
from typing import List
import nltk
import string
import pymorphy2
import codecs

In [4]:
warnings.filterwarnings(action='ignore',category=UserWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='gensim')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='smart_open') 
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='sklearn')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='scipy')    

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
data = pd.read_excel('/content/drive/MyDrive/tenderhack/dataset_base.xlsx')
data.head()

,Идентификатор СТЕ,Наименование СТЕ,Категория,Описание,Характеристики СТЕ,Регионы поставки,Кол-во заключенных контрактов,Поставщики,Страна происхождения,Другая продукция в контрактах,Просмотры,Идентификатор КПГЗ,Код КПГЗ,Модель,Цена
0,20528973,"Светодиодная лента 2835, ULTRA, 12 Вольт, 60 l...",Лампы светодиодные,NaN,"[{""Name"":""Цвет свечения"",""Id"":-506425117,""Valu...","[{""Name"":""Москва""}]",NaN,"[{""SupplierId"":2310783,""Name"":""ООО \""ЛЮМЕНВАТТ...",NaN,NaN,1,12106442.0,01.10.05.02.04,2835ULTRA,NaN
1,34860341,Светодиодный светильник Geniled ЛПО 1200?180?...,Настенно-потолочные светильники,NaN,"[{""Name"":""Вид климатического исполнения"",""Id"":...","[{""Name"":""Свердловская""},{""Name"":""Москва""}]",1.0,"[{""SupplierId"":1559525,""Name"":""Общество с огра...",РОССИЯ,NaN,41,12114082.0,01.10.05.03.01.05,ЛПО 1200?180?40 40Вт 5000К Опал,NaN
2,34172198,Елка пристенно-угловая Графская премиум 210 см...,Предметы внутреннего интерьера,NaN,"[{""Name"":""Цвет"",""Id"":340085190,""Value"":""Зелены...","[{""Name"":""Москва""},{""Name"":""Все регионы""}]",NaN,"[{""SupplierId"":1348988,""Name"":""ИП Чепало Оксан...",NaN,NaN,5,68093.0,01.20.08,Елка пристенно-угловая Графская премиум 210 см.,NaN
3,1318014,Картридж HP CE278A/Canon 728 LJ P1560/1566/160...,Расходные материалы и комплектующие для лазерн...,NaN,"[{""Name"":""Тип расходника"",""Id"":317455168,""Valu...","[{""Name"":""Москва""},{""Name"":""Московская""}]",9.0,"[{""SupplierId"":1111141,""Name"":""Общество с огра...",NaN,"[{""OtherSkuId"":1398583,""OtherSkuName"":""013R006...",6,12105204.0,01.13.13.01.01,SF-CE278(2K),NaN
4,1251411,Штамп самонаборный Pr.C20/3-Set 38х14 (аналог ...,Штемпели,NaN,"[{""Name"":""Наличие рамки"",""Id"":366100137,""Value...","[{""Name"":""Белгородская""},{""Name"":""Марий Эл""},{...",71.0,"[{""SupplierId"":1100019,""Name"":""ООО \""ЦТО\"""",""I...",АВСТРИЯ,"[{""OtherSkuId"":1171023,""OtherSkuName"":""Berling...",72,12120460.0,01.15.06.07,Штамп самонаборный пласт. 3стр. Pr.C20/3-Set 3...,"[{""Cost"":454.67000},{""Cost"":620.00000},{""Cost""..."


In [7]:
len(data)

200000

In [8]:
data['Категория'].value_counts()

Расходные материалы и комплектующие для лазерных принтеров и МФУ                                  6519
Ручки канцелярские                                                                                3230
Одежда специальная для защиты от общих производственных загрязнений и механических воздействий    3015
Папки пластиковые                                                                                 1994
Учебники печатные общеобразовательного назначения                                                 1951
                                                                                                  ... 
Общий простатический специфический антиген ивд, набор, иммунохемилюминесцентный анализ               1
Шприц для получения аутологичной кондиционированной сыворотки                                        1
Имплантат костного матрикса человеческого происхождения (мембраны)                                   1
Реагенты и расходные материалы для проточных цитофлуориметров "facscanto 

In [9]:
data.dtypes.value_counts()

object     11
int64       2
float64     2
dtype: int64

In [10]:
type(data['Характеристики СТЕ'][0])

str

In [11]:
import json
characts = []
for i in data['Характеристики СТЕ']:
  try:
    characts.append(json.loads(i))
  except:
    pass


In [12]:
print(characts[0])

[{'Name': 'Цвет свечения', 'Id': -506425117, 'Value': 'Холодный'}, {'Name': 'Мощность', 'Id': -506424125, 'Value': '7.00000', 'Unit': 'Вт'}, {'Name': 'Напряжение', 'Id': -506424124, 'Value': '12', 'Unit': 'В'}, {'Name': 'Яркость', 'Id': -506425116, 'Value': '760', 'Unit': 'лм'}, {'Name': 'Плотность led на метр', 'Id': -506425119, 'Value': '300', 'Unit': 'шт'}, {'Name': 'Размер светодиодов', 'Id': -506425118, 'Value': '283', 'Unit': 'smd'}]


In [13]:
all_characts = {}
charact_counter = {}
for charact_set in characts:
  for charact in charact_set:
    name = charact['Name'].strip().lower()
    if name[0] == '"':
      name = name[1:-1]
    elif name[0] == "'":
      name = name[1:]
    if name not in all_characts:
      all_characts[name] = set()
      charact_counter[name] = 0
    charact_counter[name]+=1
    try:
      all_characts[name].add(charact['Value'].strip().lower())
    except KeyError:
      pass


In [14]:
print(len(all_characts))
print(list(all_characts.keys())[:20])

57717
['цвет свечения', 'мощность', 'напряжение', 'яркость', 'плотность led на метр', 'размер светодиодов', 'вид климатического исполнения', 'вид рассеивателя', 'коэффициент мощности', 'вид товаров', 'вид энергетического оборудования', 'габаритные размеры (шхгхв)', 'световой поток', 'цветовая температура', 'гарантия', 'степень защиты', 'цвет', 'металлическая подставка', 'материал', 'вид ели']


In [15]:
print(list(sorted(all_characts.keys()))[:20])

['+частота процессора', '- заряд', '- разряд', '. рн 10% водного раствора мыла', '1  – пропанол', '1 disilet с отслаиваемой оболочкой', '1 xro силиконовый катетер  4,2 fr', '1 замес', '1 порт', '1 радиотрубка', '1 туннельная игла', '1,62', '1-й класс', '1-й тип коннектора', '10/100base-tx с поддержкой poe', '100 внутренних абонентов. расширяется до 200 с помощью dsp модуля d3', '100% арабика', '10base-t / 100base-tx / 1000base-t', '12 модулей размером', '12. способ измерения температуры']


In [16]:
good_keys = [key for key in charact_counter if charact_counter[key]>=10]
len(good_keys)

10555

In [17]:
sorted(good_keys)[:20]

['100% арабика',
 '3 в 1',
 '3d эффект',
 '3d-датчик ускорения',
 '3d-печать',
 '3g',
 '4g',
 '4g lte',
 '[in] номинальный ток',
 'access control list',
 'adf (податчик оригиналов)',
 'aoh',
 'atx',
 'av выход',
 'bec',
 'bio-средство',
 'bios',
 'bluetooth',
 'bluetooth встроенный',
 'classless inter-domain routing (cidr)']

In [18]:
for i in sorted(good_keys)[:20]:
  print(i, all_characts[i])

100% арабика {'нет', 'да'}
3 в 1 {'нет', 'да'}
3d эффект {'эффект движения', 'объемная'}
3d-датчик ускорения {'нет', 'да'}
3d-печать {'объемная', 'объмная с движением'}
3g {'нет', 'да'}
4g {'нет', 'да'}
4g lte {'нет', 'да'}
[in] номинальный ток {'63', '125', '16', '10', '40', '32', '1', '9', '2', '250', '80', '25', '160', '7', '315', '100', '95', '6'}
access control list {'нет', 'поддерживается, до 50 списков доступа'}
adf (податчик оригиналов) {'dadf  на 50 листов', 'нет', 'односторонний', 'наличие', '1', 'двусторонний', 'да', 'есть', 'обычный'}
aoh {'да', 'есть'}
atx {'нет', 'да'}
av выход {'нет', 'да'}
bec {'30.00000', '5.00000', '10.00000', '15', '3.60000', '3.00000', '4.00000', '1.00000', '15.00000', '20.00000', '16.00000', '40.00000', '25.00000', '52.00000', '18.00000', '8.00000'}
bio-средство {'нет', 'да'}
bios {'efi ami bios, 2x 128 мбит', 'uefi ami bios, 2x 128 мбит', 'efi ami, 64 мбит x2 с поддержкой dualbios', 'efi ami bios, 128 мбит', 'efi ami bios, 1x 128 мбит', '128 mb fl

In [19]:
data_dict = {}
for iterr, st in data.iterrows():
  data_dict[st['Идентификатор СТЕ']] = st

In [20]:
connected = {}
for iterr, st in data.iterrows():
  if st['Категория'] not in connected:
    connected[st['Категория']] = {}
  if not pd.isna(st['Другая продукция в контрактах']) and len(st['Другая продукция в контрактах'].strip())>0:
    st_others = st['Другая продукция в контрактах']
    edge_inda = -st_others[::-1].index("}")
    if edge_inda!=0:
      st_others = st_others[:edge_inda]+']'
    else:
      st_others += ']'
    # print(st_others[-20:])
    st_data = json.loads(st_others)
    for prod in st_data:
      try:
        connected_cat = data_dict[prod['OtherSkuId']]['Категория']
        if connected_cat not in connected[st['Категория']]:
          connected[st['Категория']][connected_cat] = 0
        connected[st['Категория']][connected_cat] += 1
      except KeyError:
        pass

      

In [21]:
print(len(connected))

3810


In [22]:
list(connected.keys())[:10]

['Лампы светодиодные',
 'Настенно-потолочные светильники',
 'Предметы внутреннего интерьера',
 'Расходные материалы и комплектующие для лазерных принтеров и МФУ',
 'Штемпели',
 'Блокнот',
 'Знаки различия наплечные, нарукавные, кокарды, нашивки',
 'Грамоты',
 'Салфетки, насадки для уборки, полотна технические, ветошь',
 'Флаги, вымпелы, прочая флажная продукция']

In [23]:
connected['Блокнот']

{'Data-кабели': 7,
 'USB-накопители твердотельные (флеш-драйвы)': 23,
 'Автомобильная косметика': 2,
 'Аксессуары для банка и инкассации': 56,
 'Аксессуары для кофеварок и кофемашин': 1,
 'Альбомы для рисования': 20,
 'Альбомы для фотографий': 1,
 'Антисептики кожные': 10,
 'Аптечки медицинские': 1,
 'Баннерные декорации': 2,
 'Батареи аккумуляторные никель-металл-гибридные': 8,
 'Бейджи': 151,
 'Бинты марлевые медицинские': 4,
 'Бирки для ключей': 53,
 'Бланк бухгалтерский': 43,
 'Бланк самокопирующий': 1,
 'Бланки (кроме бухгалтерских)': 46,
 'Блоки бумажные для записей': 3011,
 'Блокнот': 2667,
 'Боксы для бумаг пластиковые': 26,
 'Брошюраторы (кроме брошюраторов полиграфических)': 15,
 'Булавки': 72,
 'Бумага для офисной техники белая': 251,
 'Бумага для офисной техники цветная': 612,
 'Бумага для плоттеров и копировальных работ': 26,
 'Бумага для рисования': 41,
 'Бумага для черчения (ватман)': 146,
 'Бумага копировальная': 10,
 'Бумага мелованная для печати': 6,
 'Бумага миллимет

In [24]:
connected_keys = list(connected.keys())
num_cat = enumerate(connected_keys)
decoder = {category:i for i, category in num_cat}
print(decoder['Блокнот'])

5


In [25]:
matr = [[0 for i in range(len(connected))] for j in range(len(connected))]

In [63]:
V = [[] for i in range(len(connected))]
for category1 in connected:
  helpers = sorted(list(connected[category1].keys()), key = lambda x: -connected[category1][x])
  if helpers and helpers[0] == category1:
    helpers = helpers[1:]
  for category2 in helpers[:3]:
        matr[decoder[category1]][decoder[category2]] = connected[category1][category2]
        V[decoder[category1]].append(decoder[category2])

In [64]:
matr_df = pd.DataFrame(matr, columns = connected_keys, index = connected_keys)

In [65]:
matr_df.head()

,Лампы светодиодные,Настенно-потолочные светильники,Предметы внутреннего интерьера,Расходные материалы и комплектующие для лазерных принтеров и МФУ,Штемпели,Блокнот,"Знаки различия наплечные, нарукавные, кокарды, нашивки",Грамоты,"Салфетки, насадки для уборки, полотна технические, ветошь","Флаги, вымпелы, прочая флажная продукция",Принадлежности для флажной продукции,Бланк бухгалтерский,Ворота гандбольные,Сувениры и награды,Уборы головные для взрослых,Стенды для афиш и реклам роллерные,"Оборудование и инвентарь для футбола, мини-футбола прочее",Визитницы пластиковые,Оборудование и инвентарь для баскетбола прочее,Канцелярские журналы,Печати круглые самонаборные,Бланки (кроме бухгалтерских),Сетки-ворота,Ежедневники,"Мешки, чехлы, пакеты, сумки полимерные",Дипломы,Одежда специальная для защиты от общих производственных загрязнений и механических воздействий,Футболки,Ручки канцелярские,"Стенды, таблички, карманы информационные",Бумага для офисной техники белая,Календари бумажные,Посуда кухонная пластмассовая,Стойки волейбольные,Системы демонстрационные,Экипировка,Спортивные мешки,Системы контроля доступа,Изделия столовые из нержавеющей стали,"Набор пишущих принадлежностей, включающие в себя ручку",...,"Реагенты и расходные материалы для анализаторов гематологических ""мек-5208к""/""5216к"", ""мек-6318к"", ""мек-6400к""/""6410к"", ""мек-6500к""/""6510к"", ""мек-7222к""/""7300к"", ""мек-8222к""","Игла для биопсии мягких тканей, одноразового использования","Пробирка вакуумная для взятия образцов крови ивд, с na2эдта и натрия фторидом","Упаковочный термозапаиваемый рулонный материал для использования в низкотемпературных плазменных стерилизаторах ""sterrad""","Подкладка медицинская текстильная общего назначения, нестерильная",Анализаторы параметров стимуляции имплантируемых устройств,"Treponema pallidum антигены ивд, набор, иммунохроматографический анализ, экспресс-анализ",Маски и трубки,Установки стоматологические,Расходные материалы для очков корригирующих,Кардиостимуляторы совместимые с магнитно-резонансной томографией (мрт),Стержень костный ортопедический,Автоклавы для наконечников,Цифровые плееры,Картофель переработанный,Резектоскопы монополярные,Камины и печи,"Учебно-методические пособия, закупаемые в рамках проекта «техносфера современной школы»",Плетеная нерассасывающаяся шелковая нить (эквиваленты: шелк / шелк плетёный atramat),"Наборы ремонтные для катетеров ""broviac"" для детей","Ножка эндопротеза бедренной кости с ""пресс-фит"" фиксацией",Фартук для защиты от излучения,Костюм изолирующий,Электроды нейтральные,Чепчики детские,"Повязка для абсорбции экссудата, не гелевая, антибактериальная, стерильная",Масла технологические,Трость опорная,Пояса с петлями для обучения плаванию,Брюки для детей,Ортез запястья и кисти,Перекладины гимнастические,Имплантаты сетчатые,Герпесвирусные инфекции методом полимеразной цепной реакции (пцр) в режиме реального времени для планшетного и роторного типов,"Ревматоидный фактор ивд, набор, иммуноферментный анализ (ифа)",Игла для переноса проб крови ивд,"Упаковка для стерилизации, одноразового использования","Набор для проведения общехирургических процедур, не содержащий лекарственные средства, многоразового использования","Скрытая кровь в кале ивд, набор, иммунохроматографический анализ, экспресс-анализ",Проводники для малоинвазивной хирургии
Лампы светодиодные,0,1680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Настенно-потолочные светильники,1680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Предметы внутреннего интерьера,0,0,0,0,0,0,0,0,70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,63,0,0,0,0,0,0,0,24,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Расходные материалы и комплектующие для лазерных принтеров и МФУ,0,0,0,0,0,0,0,0,0,

In [66]:
matr_df.to_csv('/content/drive/MyDrive/tenderhack/categoty_matr_top3.csv')

In [67]:
Visited = [False] * len(connected)
def DFS(start, number):
    Visited[start] = number
    for v in V[start]:
        if not Visited[v]:
            DFS(v, number)
ncomp = 0
for i in range(0, len(connected)): 
    if not Visited[i]:
        ncomp += 1
        DFS(i, ncomp)

In [68]:
ncomp

2979

In [69]:
components = [[] for i in range(ncomp)]
for i in range(len(Visited)):
  # print(Visited[i])
  # print(components[Visited[i]])
  # print(connected_keys[i])
  components[Visited[i]-1].append(connected_keys[i])


In [70]:
summ = 0
for i in components:
  summ+=len(i)
  if len(i)>1:
    print(len(i), i)

7 ['Лампы светодиодные', 'Настенно-потолочные светильники', 'Комплектующие для кабельных изделий', 'Арматура электротехническая прочая', 'Кабель силовой с медной жилой на напряжение до 1 КВт', 'Люминесцентные лампы', 'Розетки штепсельные бытового назначения']
39 ['Предметы внутреннего интерьера', 'Ткани прочие', 'Клеи', 'Смесители', 'Краски для рисования', 'Материалы для лепки и скульптуры', 'Валики малярные', 'Кисти художественные', 'Оборудование для детских площадок прочее', 'Кисти', 'Болты, шайбы, гайки', 'Колеровочные пасты', 'Канализационные трубы', 'Наборы игровые, кроме конструкторов', 'Материалы для живописи и графики', 'Дюбеля', 'Эмали', 'Альбомы для рисования', 'Оборудование для дворовых территорий прочее', 'Фурнитура для сантехнического оборудования', 'Краски на основе акриловых и виниловых полимеров в водной среде', 'Арматура сантехническая прочая', 'Полимерные строительные материалы', 'Винты, шурупы', 'Детали трубопроводов из полиэтилена', 'Прочие товары для детей (кроме о